# train to get the best weighting matrix

In [1]:
import numpy as np
import scipy.io as sio
from scipy.optimize import minimize

In [2]:
data=sio.loadmat('ex4data1.mat')
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'X', 'y'])

In [3]:
X=data['X']
y=data['y']

In [4]:
X=np.insert(X,0,1,1)
y=y.flatten()

In [5]:
print(X.shape,y.shape)

(5000, 401) (5000,)


# y one-hot encoder

why using one-hot encode?

cost function is th cost function in logistic regression, only has two values:(0,1). Using one-hot encoder to change the y as a binary-classification

In [6]:
def onehot(y):
    result=[]
    for i in y:
        res=np.zeros(10)
        res[i-1]=1
        result.append(res)
    return np.array(result)

In [7]:
y=onehot(y)
print(y[5:])

[[0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]]


# 序列化权重，以在minimize函数中使用，minimize中，theta的size是一维

In [8]:
theta=sio.loadmat('ex4weights.mat')
theta1=theta['Theta1']
theta2=theta['Theta2']

In [9]:
def serialize(a,b):
    
    return np.append(a.flatten(),b.flatten())

In [10]:
theta_seria=serialize(theta1,theta2)
theta_seria.shape

(10285,)

In [11]:
def de_serialize(a):
    theta1=a[:25*401].reshape(25,401)
    theta2=a[25*401:].reshape(10,26)
    return theta1,theta2

In [12]:
theta1_re,theta2_re=de_serialize(theta_seria)
theta1.shape

(25, 401)

# feedforward

In [13]:
def feeedforward(theta_seria,x):
    theta1,theta2 = de_serialize(theta_seria)
    a1 = X
    z2 = a1 @ theta1.T
    a2 = sigmoid(z2)
    a2 = np.insert(a2,0,values=1,axis=1)
    z3 = a2 @ theta2.T
    h = sigmoid(z3)
    return a1,z2,a2,z3,h


In [14]:
def sigmoid(z):
    h=1/(1+np.exp(-z))
    return h

# cost function

In [15]:
# non-regulizer
def non_costFun(theta_seria,x,y):
    a1,z2,a2,z3,h=feeedforward(theta_seria,X)
    inner=y*np.log(h)+(1-y)*np.log(1-h)
    cost=-np.sum(inner)/len(x)
    return cost

In [16]:
def costFun(theta_seria,x,y,lam):
    a1,z2,a2,z3,h=feeedforward(theta_seria,X)
    theta1,theta2=de_serialize(theta_seria)
    reg=(np.sum(np.power(theta1[:,1:],2))+np.sum(np.power(theta2[:,1:],2)))*lam/(2*len(y))
    cost=reg+non_costFun(theta_seria,x,y)
    return cost


In [17]:
a1,z2,a2,z3,h=feeedforward(theta_seria,X)

In [18]:
non_costFun(theta_seria,X,y)

0.2876291651613189

# back forward

In [19]:
def sigmoid_deriv(x):
    return sigmoid(x)*(1-sigmoid(x))


In [20]:
def gradient(theta_seria,X,y):
    theta1,theta2 = de_serialize(theta_seria)
    a1,z2,a2,z3,h = feeedforward(theta_seria,X)
    d3 = h - y
    d2 = d3 @ theta2[:,1:] * sigmoid_deriv(z2)
    D2 = (d3.T @ a2) / len(X)
    D1 = (d2.T @ a1) / len(X)
    return serialize(D1,D2)

In [21]:
def non_back(theta_seria,X,y):
    theta1,theta2=de_serialize(theta_seria)
    a1,z2,a2,z3,h=feeedforward(theta_seria,X)
    d3=h-y
    d2=d3@theta2[:,1:]*sigmoid_deriv(z2)
    D2=(d3.T@a2)/len(y)
    D1=(d2.T@a1)/len(y)
    
    return serialize(D1,D2)

In [22]:
def back(theta_seria,X,y,lam):
    theta1,theta2=de_serialize(theta_seria)
    a1,z2,a2,z3,h=feeedforward(theta_seria,X)
    d3=h-y
    d2=d3@theta2[:,1:]*sigmoid_deriv(z2)
    D2=(d3.T@a2)/len(y)
    D1=(d2.T@a1)/len(y)
    reg1=theta1[:,1:]*lam/len(y)
    reg2=theta2[:,1:]*lam/len(y)
    D1[:,1:]=D1[:,1:]+reg1
    D2[:,1:]=D2[:,1:]+reg2
    return serialize(D1,D2)

# minimize

In [40]:
def optim(X,y):
    init_theta=np.random.uniform(-0.5,0.5,10285)
    res=minimize(fun=costFun,x0=init_theta,args=(X,y,10),method='TNC',jac=back,options={'maxiter':300})
    return res

In [24]:
def nn_training(X,y):
    
    init_theta = np.random.uniform(-0.5,0.5,10285)
    res = minimize(fun =non_costFun,
                   x0 = init_theta,
                  args = (X,y),
                  method='TNC',
                  jac = gradient,
                options = {'maxiter':300})
    
    return res

In [41]:
pred=optim(X,y)

In [42]:
#feedforwar
a1_pred,z2_pred,a2_pred,z3_pred,h_pred=feeedforward(pred.x,X)

In [43]:
y_pred=np.argmax(h_pred,axis=1)+1

In [44]:
yy=data['y'].flatten()
np.mean(y_pred==yy)

0.9372

In [29]:
res = nn_training(X,y)
raw_y = data['y'].reshape(5000,)
_,_,_,_,h = feeedforward(res.x,X)
y_pred = np.argmax(h,axis=1) + 1
np.mean(y_pred == raw_y)

0.9998

In [30]:
np.random.uniform(5,15,10)

array([11.96408607,  6.87012188, 13.99351738,  9.03179753,  8.3676327 ,
       10.11442836,  9.87252919, 12.97652552,  8.54608398,  9.09936878])

In [31]:
np.random.randn(10)*(2*0.1)-0.1

array([-0.25207663, -0.35277748, -0.12581215, -0.24763263, -0.32768069,
       -0.02135064, -0.24292418, -0.24080126,  0.07261381, -0.45460114])

In [32]:
y_pred[4995:]

array([9, 9, 9, 9, 9], dtype=int64)

In [33]:
y[1001]

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.])